In [14]:
## Load relevant R packages.

inLibraries = list('repr','rgdal','rgeos','prettymapr')

for (rpack in inLibraries) {
  if (is.element(rpack,installed.packages()[,1])){
      #Load the library into R
      suppressMessages(library(rpack,character.only = TRUE))
    }
    else {
        print(paste("Warning:  ",rpack," is not an installed package"))
    }
}

## Resize plot area.

options(repr.plot.width=6, repr.plot.height=6)

In [15]:

counties <- readOGR("cb_2018_us_county_500k.shp")
states <- readOGR("cb_2018_us_state_500k.shp")


OGR data source with driver: ESRI Shapefile 
Source: "cb_2018_us_county_500k.shp", layer: "cb_2018_us_county_500k"
with 3233 features
It has 9 fields


NOTE: rgdal::checkCRSArgs: no proj_defs.dat in PROJ.4 shared files


OGR data source with driver: ESRI Shapefile 
Source: "cb_2018_us_state_500k.shp", layer: "cb_2018_us_state_500k"
with 56 features
It has 9 fields


NOTE: rgdal::checkCRSArgs: no proj_defs.dat in PROJ.4 shared files


In [ ]:
stateselectfp <- c("48","08","29")
statessel <- states[states@data$STATEFP %in% stateselectfp,]
countiessel <- counties[counties@data$STATEFP %in% stateselectfp,]
countiessel@data$STATENM[countiessel@data$STATEFP == "08"] <- "Colorado"
countiessel@data$STATENM[countiessel@data$STATEFP == "48"] <- "Texas"
countiessel@data$STATENM[countiessel@data$STATEFP == "29"] <- "Missouri"

In [16]:
df <- read.csv("co-est2019-alldata.csv")
library(stringr)
df$GEOID <- paste0(str_pad(df$STATE,2,pad="0"),str_pad(df$COUNTY,3,pad="0"))
df$GEOIDST <- paste0(str_pad(df$STATE,2,pad="0"))
popest2019 <- df[,c("STNAME","CTYNAME","POPESTIMATE2019", "GEOID","GEOIDST")]
#popest2019 <- popest2019[popest2019$STNAME %in% c("Colorado","Texas","Missouri"),]
#stpop2019 <- popest2019[popest2019$CTYNAME %in% c("Colorado","Texas","Missouri"),]
#copop2019 <- popest2019[!popest2019$CTYNAME %in% c("Colorado","Texas","Missouri"),]

In [10]:
stpop2019 <- popest2019[as.character(popest2019$STNAME) == as.character(popest2019$CTYNAME),][,c("POPESTIMATE2019","GEOID","GEOIDST")]
copop2019 <- popest2019[!as.character(popest2019$STNAME) == as.character(popest2019$CTYNAME),][,c("STNAME","POPESTIMATE2019","GEOID")]
stpop2019 <- stpop2019[stpop2019$GEOID != "11001",]
stpop2019$GEOID <- stpop2019$GEOIDST
stpop2019 <- stpop2019[,c("POPESTIMATE2019","GEOID")]


In [17]:
states <- merge(states,stpop2019,by="GEOID")
counties <- merge(counties,copop2019,by="GEOID")

In [18]:
head(counties)

An object of class "SpatialPolygonsDataFrame"
Slot "data":
     GEOID STATEFP COUNTYFP COUNTYNS       AFFGEOID    NAME LSAD      ALAND
997  21007      21      007 00516850 0500000US21007 Ballard   06  639387454
1002 21017      21      017 00516855 0500000US21017 Bourbon   06  750439351
1009 21031      21      031 00516862 0500000US21031  Butler   06 1103571974
1026 21065      21      065 00516879 0500000US21065  Estill   06  655509930
1028 21069      21      069 00516881 0500000US21069 Fleming   06  902727151
1040 21093      21      093 00516893 0500000US21093  Hardin   06 1614569777
       AWATER   STNAME POPESTIMATE2019
997  69473325 Kentucky            7888
1002  4829777 Kentucky           19788
1009 13943044 Kentucky           12879
1026  6516335 Kentucky           14106
1028  7182793 Kentucky           14581
1040 17463238 Kentucky          110958

Slot "polygons":
[[1]]
An object of class "Polygons"
Slot "Polygons":
[[1]]
An object of class "Polygon"
Slot "labpt":
[1] -88.99926  3

In [19]:
writeOGR(counties, "counties_geojson", layer="countiessel", driver="GeoJSON")
writeOGR(states, "states_geojson", layer="statessel", driver="GeoJSON")